In [2]:
import numpy as np
import pandas as pd
import geopandas as gpd
import warnings
from shapely.geometry import Polygon
import sys
import os

In [3]:
thresholds = [300, 600, 1000]

In [4]:
warnings.filterwarnings('ignore')

path = "C:/Dumps/GEE-WP Scores/Gravity/100m grids/Grid_geoms/"
ls_file = os.listdir(path)
ls_file = np.array(ls_file)[[i.rsplit('_')[1] == 'grav3' for i in ls_file]]

cities_scores = list()
for i in ls_file:
    city_score = gpd.read_file(path+i)
    scores_df = city_score.iloc[range(0,len(city_score),3)]['geometry'].reset_index(drop = True)
    for j in thresholds:
        ls = list()
        score = city_score['tr_'+str(j)].dropna().reset_index(drop =True)
        for k in score:
            if k == 0: 
                ls.append('0 no')
            elif k < j: # one perfect score (no route cost to UGS); score is threshold - route (and entry) cost
                ls.append('1 low')
            elif k < j*2: # two perfect scores
                ls.append('2 mediocre')
            elif k < j*3: # three perfect scores
                ls.append('3 sufficient')
            elif k < j*4: # four perfect scores
                ls.append('4 good')
            else: # above four perfect scores (or 8 UGS on half the threshold distance)
                ls.append('5 excellent')
        print(i.rsplit('_')[2].rsplit('.')[0], j)
        scores_df = pd.concat([scores_df, 
                               pd.Series(round(score, 1), name = 'Grav-Sc '+str(j)), 
                               pd.Series(ls, name = 'GravQnt '+str(j))], 
                              axis = 1)
    cities_scores.append(scores_df)
cities_scores[0]

Addis Ababa 300
Addis Ababa 600
Addis Ababa 1000
Dhaka 300
Dhaka 600
Dhaka 1000
Santo Domingo 300
Santo Domingo 600
Santo Domingo 1000
Shijiazhuang 300
Shijiazhuang 600
Shijiazhuang 1000


,geometry,Grav-Sc 300,GravQnt 300,Grav-Sc 600,GravQnt 600,Grav-Sc 1000,GravQnt 1000
0,"POLYGON ((38.63958 9.09875, 38.64042 9.09875, ...",0.0,0 no,0.0,0 no,0.0,0 no
1,"POLYGON ((38.64042 9.09875, 38.64125 9.09875, ...",0.0,0 no,0.0,0 no,0.0,0 no
2,"POLYGON ((38.64792 9.09875, 38.64875 9.09875, ...",0.0,0 no,0.0,0 no,0.0,0 no
3,"POLYGON ((38.72292 9.09875, 38.72375 9.09875, ...",300.0,2 mediocre,600.0,2 mediocre,1000.0,2 mediocre
4,"POLYGON ((38.72375 9.09875, 38.72458 9.09875, ...",300.0,2 mediocre,600.0,2 mediocre,1000.0,2 mediocre
...,...,...,...,...,...,...,...
101324,"POLYGON ((38.71875 9.01625, 38.71958 9.01625, ...",0.0,0 no,0.0,0 no,0.0,0 no
101325,"POLYGON ((38.71958 9.01625, 38.72042 9.01625, ...",0.0,0 no,0.0,0 no,280.7,1 low
101326,"POLYGON ((38.72042 9.01625, 38.72125 9.01625, ...",0.0,0 no,78.1,1 low,478.1,1 low
101327,"POLYGON ((38.72125 9.01625, 38.72208 9.01625, ...",0.0,0 no,0.0,0 no,301.6,1 low


In [43]:
E2SCFA_class = list()
path = "C:/Dumps/GEE-WP Scores/E2SFCA/100m grids/grid_geoms/"
ls_file = os.listdir(path)
for i in ls_file:
    city_score = gpd.read_file(path+i)
    score_df = city_score['population']
    for j in thresholds:
        ls = list()
        score = city_score['Sc-norm '+str(j)]
        for k in score:
            if k == 0: # m2 of UGS per person and weighted with Gaussian (threshold capped) distance decay
                ls.append('0 no')
            elif k < 9: # m2 per person required by the WHO
                ls.append('1 low')
            elif k < 18: # m2 per person twice required by the WHO
                ls.append('2 mediocre')
            elif k < 30: # m2 per person preferred by the WHO
                ls.append('3 sufficient')
            elif k < 100: # over 100m2 per person, the additive value of m2 per person becomes almost obsolete
                ls.append('4 good') # below the 100m2 is still a good score
            else:
                ls.append('5 excellent') # above it will be excellent
        print(i.rsplit('.')[0], j)
        score_df = pd.concat([score_df, 
                              round(pd.Series(score, name = 'E2SCFA-Sc '+str(j)),2), 
                              pd.Series(ls, name = 'E2SCFA-class '+str(j))], 
                             axis = 1)
    E2SCFA_class.append(score_df)
E2SCFA_class[0]

Addis Ababa 300
Addis Ababa 600
Addis Ababa 1000
Damascus 300
Damascus 600
Damascus 1000
Dhaka 300
Dhaka 600
Dhaka 1000
Shijiazhuang 300
Shijiazhuang 600
Shijiazhuang 1000


,population,E2SCFA-Sc 300,E2SCFA-class 300,E2SCFA-Sc 600,E2SCFA-class 600,E2SCFA-Sc 1000,E2SCFA-class 1000
0,3,0.00,0 no,0.00,0 no,0.00,0 no
1,3,0.00,0 no,0.00,0 no,0.00,0 no
2,3,0.00,0 no,0.00,0 no,0.00,0 no
3,5,266.72,5 excellent,207.06,5 excellent,154.27,5 excellent
4,5,266.72,5 excellent,207.06,5 excellent,154.27,5 excellent
...,...,...,...,...,...,...,...
101324,217,0.00,0 no,0.00,0 no,0.00,0 no
101325,229,0.00,0 no,0.00,0 no,0.00,0 no
101326,234,0.00,0 no,0.00,0 no,0.00,0 no
101327,231,0.00,0 no,0.00,0 no,0.00,0 no


In [38]:
scores = list()
write_path = 'D:/Dumps/GEE-WP Scores/city_summary/'
for i in enumerate(cities):
    score = pd.concat([E2SCFA_class[i[0]], cities_scores[i[0]]], axis = 1)
    
    score = score[['geometry','population',
                   'E2SCFA-class 300','E2SCFA-class 600','E2SCFA-class 1000',
                   'GravQnt 300','GravQnt 600','GravQnt 1000',
                   'E2SCFA-Sc 300','E2SCFA-Sc 600','E2SCFA-Sc 1000',
                   'Grav-Sc 300','Grav-Sc 600','Grav-Sc 1000']]
        
    # Adding categories without grid geoms attached to ensure that each category is in a plotted map
    ls2 = pd.DataFrame(list([
                [Polygon(),'5 excellent','5 excellent','5 excellent','5 excellent','5 excellent','5 excellent'],
                [Polygon(),'4 good','4 good','4 good','4 good','4 good','4 good'],
                [Polygon(),'3 sufficient','3 sufficient','3 sufficient','3 sufficient','3 sufficient','3 sufficient'],
                [Polygon(),'2 mediocre','2 mediocre','2 mediocre','2 mediocre','2 mediocre','2 mediocre'],
                [Polygon(),'1 low','1 low','1 low','1 low','1 low','1 low'],
                [Polygon(),'0 no','0 no','0 no','0 no','0 no','0 no']]))
    
    ls2.columns = ['geometry',
                   'E2SCFA-class 300','E2SCFA-class 600','E2SCFA-class 1000',
                   'GravQnt 300','GravQnt 600','GravQnt 1000']
    
    score = pd.concat([score, pd.DataFrame(ls2)])
    
    score = gpd.GeoDataFrame(score, geometry = 'geometry', crs = 4326)
    score.to_file(write_path+i[1]+'.gpkg')
    score.iloc[:,score.columns != 'geometry'].to_csv(write_path+i[1]+'.csv')
    scores.append(score)

In [39]:
[[print(city[1], 
        scores[city[0]].groupby('E2SCFA-class '+str(t))['E2SCFA-class '+str(t)].count()) 
  for t in thresholds] 
 for city in enumerate(cities)]
sys.stdout.flush()

Dhaka E2SCFA-class 300
0 no            94317
1 low            2525
2 mediocre        421
3 sufficient      741
4 good           1918
5 excellent      1413
Name: E2SCFA-class 300, dtype: int64
Dhaka E2SCFA-class 600
0 no            91269
1 low            5199
2 mediocre        862
3 sufficient      918
4 good           1792
5 excellent      1295
Name: E2SCFA-class 600, dtype: int64
Dhaka E2SCFA-class 1000
0 no            87420
1 low            8866
2 mediocre       1325
3 sufficient     1009
4 good           1559
5 excellent      1156
Name: E2SCFA-class 1000, dtype: int64
Damascus E2SCFA-class 300
0 no            21514
1 low            4655
2 mediocre          1
3 sufficient        1
4 good              1
5 excellent         1
Name: E2SCFA-class 300, dtype: int64
Damascus E2SCFA-class 600
0 no            19149
1 low            7020
2 mediocre          1
3 sufficient        1
4 good              1
5 excellent         1
Name: E2SCFA-class 600, dtype: int64
Damascus E2SCFA-class 1000
0 no 

In [46]:
scores[0][0:10]

,geometry,population,E2SCFA-class 300,E2SCFA-class 600,E2SCFA-class 1000,GravQnt 300,GravQnt 600,GravQnt 1000,E2SCFA-Sc 300,E2SCFA-Sc 600,E2SCFA-Sc 1000,Grav-Sc 300,Grav-Sc 600,Grav-Sc 1000
0,"POLYGON ((38.63958 9.09875, 38.64042 9.09875, ...",3.0,0 no,0 no,0 no,0 no,0 no,0 no,0.00,0.00,0.00,0.0,0.0,0.0
1,"POLYGON ((38.64042 9.09875, 38.64125 9.09875, ...",3.0,0 no,0 no,0 no,0 no,0 no,0 no,0.00,0.00,0.00,0.0,0.0,0.0
2,"POLYGON ((38.64792 9.09875, 38.64875 9.09875, ...",3.0,0 no,0 no,0 no,0 no,0 no,0 no,0.00,0.00,0.00,0.0,0.0,0.0
3,"POLYGON ((38.72292 9.09875, 38.72375 9.09875, ...",5.0,5 excellent,5 excellent,5 excellent,2 mediocre,2 mediocre,2 mediocre,266.72,207.06,154.27,300.0,600.0,1000.0
4,"POLYGON ((38.72375 9.09875, 38.72458 9.09875, ...",5.0,5 excellent,5 excellent,5 excellent,2 mediocre,2 mediocre,2 mediocre,266.72,207.06,154.27,300.0,600.0,1000.0
5,"POLYGON ((38.72458 9.09875, 38.72542 9.09875, ...",4.0,5 excellent,5 excellent,5 excellent,2 mediocre,2 mediocre,2 mediocre,333.40,258.82,192.84,300.0,600.0,1000.0
6,"POLYGON ((38.72542 9.09875, 38.72625 9.09875, ...",4.0,0 no,0 no,0 no,2 mediocre,2 mediocre,2 mediocre,0.00,0.00,0.00,300.0,600.0,1000.0
7,"POLYGON ((38.72625 9.09875, 38.72708 9.09875, ...",4.0,0 no,0 no,0 no,2 mediocre,2 mediocre,2 mediocre,0.00,0.00,0.00,300.0,600.0,1000.0
8,"POLYGON ((38.72708 9.09875, 38.72792 9.09875, ...",3.0,0 no,0 no,0 no,2 mediocre,2 mediocre,2 mediocre,0.00,0.00,0.00,300.0,600.0,1000.0
9,"POLYGON ((38.72792 9.09875, 38.72875 9.09875, ...",3.0,0 no,0 no,0 no,2 mediocre,2 mediocre,2 mediocre,0.00,0.00,0.00,300.0,600.0,1000.0
